In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

LOADING NOTES EXTRACTED FROM MIDI FILE IN GEERATING_NOTES-FORM_MIDI_FILE.PY FILE

In [ ]:
file='/content/drive/MyDrive/my_new_notes_27_07_2021.pkl'
fileobj=open(file,'rb')
data=pickle.load(fileobj)

In [ ]:
# data=data[:20000]
len(data)

499499

In [ ]:
data=data[:100000]

CALCULATNG THE VOCAB_SIZE AND THE DATA_SIZE

In [ ]:
characters=list(set(data))
data_size,vocab_size=len(data),len(characters)
print("The size of data is ")
print(data_size)
print("Vocab size")
print(vocab_size)

The size of data is 
100000
Vocab size
416


In [ ]:
chars = sorted(characters)
print(chars)

['0', '0.1', '0.1.2', '0.1.4', '0.1.5', '0.1.5.7', '0.1.5.8', '0.2', '0.2.3', '0.2.4', '0.2.4.6', '0.2.4.6.7', '0.2.4.7', '0.2.4.7.9', '0.2.5', '0.2.5.7', '0.2.6', '0.2.6.8', '0.2.7', '0.3', '0.3.4', '0.3.5', '0.3.5.8', '0.3.6', '0.3.6.8', '0.3.6.9', '0.3.7', '0.4', '0.4.5', '0.4.5.7', '0.4.6', '0.4.6.7', '0.4.7', '0.4.7.8', '0.4.8', '0.5', '0.5.6', '0.6', '1', '1.2', '1.2.3', '1.2.4', '1.2.4.6.8', '1.2.7', '1.3', '1.3.5.8', '1.3.6.9', '1.3.7', '1.3.7.9', '1.3.8', '1.4', '1.4.5', '1.4.6', '1.4.6.9', '1.4.7', '1.4.7.10', '1.4.7.9', '1.4.8', '1.5', '1.5.7', '1.5.8', '1.5.9', '1.6', '1.6.7', '1.7', '10', '10.0', '10.0.2', '10.0.3', '10.0.3.5', '10.0.4', '10.0.5', '10.1', '10.1.2', '10.1.3.6', '10.1.4', '10.1.4.6', '10.1.5', '10.11', '10.11.2', '10.11.2.6', '10.11.3', '10.11.4', '10.2', '10.2.3', '10.2.4', '10.2.5', '10.3', '10.3.4', '11', '11.0', '11.0.1', '11.0.4', '11.0.4.6', '11.0.4.7', '11.1', '11.1.2', '11.1.4', '11.1.4.6', '11.1.4.7', '11.1.5', '11.1.6', '11.2', '11.2.4', '11.2.4.5.

CREATING APPING FROM NOTES TO INTEGER AND THE REVERSE MAPPING

In [ ]:
char_to_ix={ch:i for i,ch in enumerate(chars)}
ix_to_char={i:ch for i,ch in enumerate(chars)}

SPLITTING THE WHOLE DATASET INTO INPUT AND OUTPUT DATASET .


In [ ]:
input=list()
output=list()
len_input=100
for i in range(len_input,data_size):
  input.append(data[i-len_input:i])
  output.append(data[i])

In [ ]:
sequences=list()
for i in input:
  enq_seq=[char_to_ix[c] for c in i]
  sequences.append(enq_seq)
# print(sequences)
outputsequeces=list()
for i in output:
  outputsequeces.append(char_to_ix[i])

CREATING ONE-HOT VETCORS FOR BOTH INPUT AND OUTPUT

In [ ]:
# from tensorflow.keras.utils import to_categorical

In [ ]:
# sequences=[to_categorical(i,num_classes=vocab_size) for i in sequences]

In [ ]:
# import numpy as np

In [ ]:
# sequences=np.array(sequences)

In [ ]:
# sequences.shape

(19900, 100, 264)

In [ ]:
# outputsequeces=[to_categorical(i,num_classes=vocab_size) for i in outputsequeces]

In [ ]:
# outputsequeces=np.array(outputsequeces)

In [ ]:
# outputsequeces.shape

(19900, 264)

CREATING MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
n_a=75

In [ ]:
import numpy as np

In [ ]:
emb_matrix=np.zeros((vocab_size,vocab_size))
for i in range(vocab_size):
  emb_matrix[i][i]=1

In [ ]:
maxlen=100

In [ ]:
import tensorflow as tf
from tensorflow.keras.initializers import Constant

In [ ]:
emb_matrix.shape

(416, 416)

In [ ]:
vocab_size

416

In [ ]:
model=tf.keras.models.Sequential([
                                tf.keras.layers.Embedding(vocab_size,vocab_size, #number of features
                                embeddings_initializer=Constant(emb_matrix), # initialize 
                                input_length=maxlen, 
                                trainable=False),
                                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_a,return_sequences=True)),
                                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_a)),
                                tf.keras.layers.Dense(vocab_size,activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# model=tf.keras.models.Sequential([
#   tf.keras.layers.Embedding(vocab_size,vocab_size, #number of features
#                                   embeddings_initializer=Constant(emb_matrix), # initialize 
#                                   input_length=maxlen, 
#                                   trainable=False),
#   LSTM(n_a,return_sequences=True),
#   LSTM(n_a),
#   Dense(vocab_size,activation='softmax')
# ])
# model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 416)          173056    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 150)          295200    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150)               135600    
_________________________________________________________________
dense (Dense)                (None, 416)               62816     
Total params: 666,672
Trainable params: 493,616
Non-trainable params: 173,056
_________________________________________________________________


TRAINGIN THE MODEL

In [ ]:
# model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(sequences,outputsequeces,epochs=100)

Epoch 1/100
3122/3122 [==============================] - 343s 106ms/step - loss: 3.3896 - accuracy: 0.1926
Epoch 2/100
3122/3122 [==============================] - 333s 107ms/step - loss: 2.7891 - accuracy: 0.2665
Epoch 3/100
3122/3122 [==============================] - 331s 106ms/step - loss: 2.5632 - accuracy: 0.3086
Epoch 4/100
3122/3122 [==============================] - 332s 106ms/step - loss: 2.3932 - accuracy: 0.3506
Epoch 5/100
3122/3122 [==============================] - 331s 106ms/step - loss: 2.2469 - accuracy: 0.3885
Epoch 6/100
3122/3122 [==============================] - 333s 107ms/step - loss: 2.1116 - accuracy: 0.4232
Epoch 7/100
3122/3122 [==============================] - 332s 106ms/step - loss: 1.9974 - accuracy: 0.4541
Epoch 8/100
3122/3122 [==============================] - 330s 106ms/step - loss: 1.8873 - accuracy: 0.4837
Epoch 9/100
3122/3122 [==============================] - 330s 106ms/step - loss: 1.7885 - accuracy: 0.5103
Epoch 10/100
3122/3122 [=============

KeyboardInterrupt: ignored

SAVING THE MODEL AND OTHER NECESSARY FILES

In [ ]:
model.save('model_music_generation_using_LSTMS.h5')

In [ ]:
from pickle import dump

In [ ]:
dump(sequences, open('sequences_music_generation_using_LSTMS.pkl', 'wb'))

In [ ]:
dump(outputsequeces, open('outputsequeces_music_generation_using_LSTMS.pkl', 'wb'))

In [ ]:
dump(char_to_ix, open('char_to_ix_music_generation_using_LSTMS.pkl', 'wb'))

In [ ]:
dump(ix_to_char, open('ix_to_char_music_generation_using_LSTMS.pkl', 'wb'))